In [1]:
import tensorflow as tf

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '15'

import numpy as np
import pandas as pd
import keras

2024-06-06 07:57:54.103188: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-06 07:57:54.370032: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 07:57:54.370127: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 07:57:54.420995: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-06 07:57:54.483093: I tensorflow/core/platform/cpu_feature_guar

In [2]:
#file_path = pd.read_csv('shahname.csv')
file_path = pd.read_csv('shahname.csv', encoding='utf-8')
file_path.head()

,Chapter,Part,Bait,Mesra,Text
0,1,1,1,1,به نام خداوند جان و خرد
1,1,1,1,2,کز این برتر اندیشه بر نگذرد
2,1,1,2,1,خداوند نام و خداوند جای
3,1,1,2,2,خداوند روزی ده رهنمای
4,1,1,3,1,خداوند کیوان و گَردان سپهر


In [3]:
column_data = file_path['Text']

combined_data = ','.join(column_data.astype(str))

In [4]:
combined_data[:101]

'به نام خداوند جان و خرد,کز این برتر اندیشه بر نگذرد,خداوند نام و خداوند جای,خداوند روزی ده رهنمای,خدا'

In [5]:
print(f'Length of text: {len(combined_data)} characters')

Length of text: 2565041 characters


In [6]:
vocab = sorted(set(combined_data))
print(f'{len(vocab)} unique characters')

57 unique characters


In [15]:
for v in vocab:
    print(f'{v}', end=' ')

  ! ( ) , :   « » ، ؛ ؟ ء آ أ ؤ ئ ا ب ت ث ج ح خ د ذ ر ز س ش ص ض ط ظ ع غ ف ق ل م ن ه و ي َ ُ ِ ّ ْ ٔ پ چ ژ ک گ ی ‌ 

In [14]:
vocab

[' ',
 '!',
 '(',
 ')',
 ',',
 ':',
 '\xa0',
 '«',
 '»',
 '،',
 '؛',
 '؟',
 'ء',
 'آ',
 'أ',
 'ؤ',
 'ئ',
 'ا',
 'ب',
 'ت',
 'ث',
 'ج',
 'ح',
 'خ',
 'د',
 'ذ',
 'ر',
 'ز',
 'س',
 'ش',
 'ص',
 'ض',
 'ط',
 'ظ',
 'ع',
 'غ',
 'ف',
 'ق',
 'ل',
 'م',
 'ن',
 'ه',
 'و',
 'ي',
 'َ',
 'ُ',
 'ِ',
 'ّ',
 'ْ',
 'ٔ',
 'پ',
 'چ',
 'ژ',
 'ک',
 'گ',
 'ی',
 '\u200c']

In [9]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in combined_data])

In [13]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ' ' :   0,
  '!' :   1,
  '(' :   2,
  ')' :   3,
  ',' :   4,
  ':' :   5,
  '\xa0':   6,
  '«' :   7,
  '»' :   8,
  '،' :   9,
  '؛' :  10,
  '؟' :  11,
  'ء' :  12,
  'آ' :  13,
  'أ' :  14,
  'ؤ' :  15,
  'ئ' :  16,
  'ا' :  17,
  'ب' :  18,
  'ت' :  19,
  ...
}


In [10]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(combined_data[:13]), text_as_int[:13]))

'به نام خداوند' ---- characters mapped to int ---- > [18 41  0 40 17 39  0 23 24 17 42 40 24]


In [17]:
seq_length = 100

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

ب
ه
 
ن
ا


In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))
    print("***"*5)

'به نام خداوند جان و خرد,کز این برتر اندیشه بر نگذرد,خداوند نام و خداوند جای,خداوند روزی ده رهنمای,خدا'
***************
'وند کیوان و گَردان سپهر,فروزندهٔ\xa0ماه و ناهید و مهر,ز نام و نشان و گمان برتر است,نگارندهٔ بر شده پیکر '
***************
'است,به بینندگان آفریننده را,نبینی مرنجان دو بیننده را,نیابد بدو نیز اندیشه راه,که او برتر از نام و از'
***************
' جایگاه,سخن هر چه زین گوهران بگذرد,نیابد بدو راه جان و خرد,خرد گر سخن برگزیند همی,همان را گزیند، که ب'
***************
'یند همی,ستودن نداند کس، او را چو هست,میان بندگی را ببایدت بست,خرد را و جان را همی سنجد اوی,در اندیشهٔ'
***************


In [13]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'به نام خداوند جان و خرد,کز این برتر اندیشه بر نگذرد,خداوند نام و خداوند جای,خداوند روزی ده رهنمای,خد'
Target data: 'ه نام خداوند جان و خرد,کز این برتر اندیشه بر نگذرد,خداوند نام و خداوند جای,خداوند روزی ده رهنمای,خدا'


In [15]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('ب')
  expected output: 41 ('ه')
Step    1
  input: 41 ('ه')
  expected output: 0 (' ')
Step    2
  input: 0 (' ')
  expected output: 40 ('ن')
Step    3
  input: 40 ('ن')
  expected output: 17 ('ا')
Step    4
  input: 17 ('ا')
  expected output: 39 ('م')


In [16]:
BATCH_SIZE = 64


dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [17]:
vocab_size = len(vocab)

embedding_dim = 25

rnn_units = 1024

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [19]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 25)            1425      
                                                                 
 gru (GRU)                   (64, None, 1024)          3228672   
                                                                 
 dense (Dense)               (64, None, 57)            58425     
                                                                 
Total params: 3288522 (12.54 MB)
Trainable params: 3288522 (12.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [22]:

model.compile(optimizer='adam', loss=loss)

In [23]:
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [24]:
history = model.fit(dataset, epochs=5, callbacks=[checkpoint_callback])

Epoch 1/5


2024-06-05 17:39:14.524870: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-06-05 17:39:14.593981: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f1fe1406fa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-05 17:39:14.594005: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX330, Compute Capability 6.1
2024-06-05 17:39:14.598417: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-05 17:39:14.624842: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
I0000 00:00:1717623554.664254    5942 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


396/396 [==============================] - 83s 205ms/step - loss: 2.5469
Epoch 2/5
396/396 [==============================] - 84s 213ms/step - loss: 1.8907
Epoch 3/5
396/396 [==============================] - 157s 396ms/step - loss: 1.6211
Epoch 4/5
396/396 [==============================] - 246s 620ms/step - loss: 1.4819
Epoch 5/5
396/396 [==============================] - 239s 603ms/step - loss: 1.3969


In [25]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_5'

In [26]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 25)             1425      
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3228672   
                                                                 
 dense_1 (Dense)             (1, None, 57)             58425     
                                                                 
Total params: 3288522 (12.54 MB)
Trainable params: 3288522 (12.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []


  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions 
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [29]:
print(generate_text(model, start_string=u"به نام خدا"))

به نام خدای سوار,که هامون بود آن جهان شهریار,ز هر گونه امرم پرشمرده شد سقل بود یاد,پسستلیده جه در زینهار,بیارم نباشد مرا کرد رای,اگر پرستیزی ز پروردگار,تو از دشمنستاشن تاهوار,همی‌دید شاه,ز من شاه هرگز نیک و نبند,چو رواشتشایاری آن پادشا را بدی,ببندی دو پرسیز و نر شاه باد,در باره گشتاسپ بر مخت ارم,مرا سر باه آن گزارین نشان,بدر بارگوها پراگنده درد,که او این سخن زرد گفتن نهان,پس از جوی بر گنج و از پادشست,همه گشته از خوب و زافون شویم,نیا مرد برنا و پر سبی بنبشیم,زبا تا به مرز گران راه جوی,کزان گوره‌آذرنگه برگرفت,بپاشید زان تخم و بوران بجست,چو از درد دیبای یزدان پاس,ز جز دادگان گوشت این کارشاه,ازین پس جهانی به خامش بغز,چو آمد به تاخندی و سور نو,چو آگاهی و خاک درمان نزد زبان,ز لشکر نبودیشاد یل ناگه کرد,همه سنکل,وه با روز آتش گناه,به خوبی فراوان سرآورده یاه,بیاره به دلش از پژندش بدام,کسی کردگار جهان زنده شاه,جهان رفت روز سیاوشیر کیه,بدو گفت گفت آنک با ا برنبد پناه دبیر,همان پهلوان یلاب آفرین,اگر تنگ شاهی که بد شاه گشت,سوی ماه خرادم بخوانمایه‌ام,یمان سر به سر بر یکی مهره اندر گلاز,که در آبیور آ